# Description

This notebook is applying simple cnn on digits dataset, achieving a 99% accuracy.

# 1. loading package and data

In [2]:
#basic package importing
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#loading data
data = pd.read_csv("train.csv").values
labels = data[:,0].astype('int32')
image = data[:,1:].astype('float32')

#kaggle test set
test_data = pd.read_csv("test.csv").values.astype('float32')

In [5]:
data.shape

(42000, 785)

# 2.Data Preparation

In [16]:
#parameters
batch_size = 128
num_classes = 10

#it takes a long time on laptop. Set epoch to 1 just for testing
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [8]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(image, labels, test_size=0.2, random_state=0)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

#normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#The kaggle test set
test_data = test_data.reshape(test_data.shape[0], img_rows, img_cols, 1)
test_data = test_data.astype('float32')
test_data /=255

# 3. Building the framework

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

In [12]:
# Almost LeNet architecture
model = Sequential()
#first convolution layer
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

#second convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [13]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

# 4. Running and Testing

In [17]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 33600 samples, validate on 8400 samples
Epoch 1/1
33600/33600 [==============================] - 105s 3ms/step - loss: 0.2514 - acc: 0.9246 - val_loss: 0.0817 - val_acc: 0.9756


In [19]:
#prediction
v = np.argmax(model.predict(test_data),1)

#output
df = pd.DataFrame(v)
df.index+=1
df.index.name='ImageId'
df.columns=['Label']
df.to_csv('results.csv', header=True)